In [ ]:

import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:

def get_dataCR():
    DB_HOST = "37.61.241.45"
    DB_PORT = "5432"
    DB_NAME = "Khubeo_IA"
    DB_USER = "postgres"
    DB_PASS = "Xjp2yCm$G36WR4E"
    
    engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    
    with engine.connect() as conn:
        result = conn.execute(text("""select * from t_compte_rendu cr
        left join t_dates da
            on da.fulldatealternatekey = cr.cr_date
        left join t_dossier dos
            on dos.id_dossier = cr.id_dossier
        left join t_enseigne en
            on en.id_enseigne = dos.do_id_enseigne
    where cr.id_client not in  (15,31,100, 101) limit 500;"""))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())  # Convert result to DataFrame
        return df

In [ ]:
def compute_hebergement_service(row):
    """
    Process financial data for accommodation and services, calculating TTC, HT, and TVA values.
    Also calculates total values across accommodation and services.
    """
    # Helper function to safely convert to float (returns original value if conversion fails)
    def safe_float(value):
        # Try to convert to float, return original value if it fails
        try:
            return float(value)
        except (TypeError, ValueError):
            return value
    
    # Helper function to calculate the three financial values (TTC, HT, TVA)
    def calculate_financial_values(ttc, ht, tva):
        # Calculate TTC if missing or zero (using HT * 1.10)
        if pd.isnull(ttc) or ttc == 0:
            ttc = ht * 1.10 if pd.notnull(ht) and ht != 0 else ttc
        
        # Calculate HT if missing or zero (using TTC / 1.10)
        if pd.isnull(ht) or ht == 0:
            ht = ttc / 1.10 if pd.notnull(ttc) and ttc != 0 else ht
        
        # Calculate TVA if missing or zero
        if pd.isnull(tva) or tva == 0:
            if pd.notnull(ht) and ht > 0:
                tva = ht * 0.10  # Calculate from HT
            elif pd.notnull(ttc) and ttc != 0:
                tva = ttc - (ttc / 1.10)  # Calculate from TTC
            # else: keep original tva
        
        return ttc, ht, tva
    
    # Extract and convert accommodation values
    hebergement_ttc = safe_float(row['cr_hebergement_ttc'])
    hebergement_ht = safe_float(row['cr_hebergement_ht'])
    hebergement_tva = safe_float(row['cr_hebergement_tva'])
    
    # Calculate accommodation financial values
    ttc_hebergement, ht_hebergement, tva_hebergement = calculate_financial_values(
        hebergement_ttc, hebergement_ht, hebergement_tva
    )
    
    # Extract and convert service values
    service_ttc = safe_float(row['cr_service_ttc'])
    service_ht = safe_float(row['cr_service_ht'])
    service_tva = safe_float(row['cr_service_tva'])
    
    # Calculate service financial values
    ttc_service, ht_service, tva_service = calculate_financial_values(
        service_ttc, service_ht, service_tva
    )
    
    # Calculate total values by adding accommodation and service
    total_ht = None
    if pd.notnull(ht_hebergement) or pd.notnull(ht_service):
        total_ht = (ht_hebergement if pd.notnull(ht_hebergement) else 0) + (ht_service if pd.notnull(ht_service) else 0)
    
    total_ttc = None
    if pd.notnull(ttc_hebergement) or pd.notnull(ttc_service):
        total_ttc = (ttc_hebergement if pd.notnull(ttc_hebergement) else 0) + (ttc_service if pd.notnull(ttc_service) else 0)
    
    total_tva = None
    if pd.notnull(tva_hebergement) or pd.notnull(tva_service):
        total_tva = (tva_hebergement if pd.notnull(tva_hebergement) else 0) + (tva_service if pd.notnull(tva_service) else 0)
    
    # Return all calculated values as a pandas Series
    return pd.Series({
        'cr_chb_dispo': row['cr_chb_dispo'],
        'cr_chb_louees': row['cr_chb_louees'],
        'cr_hebergement_ttc': ttc_hebergement,
        'cr_hebergement_ht': ht_hebergement,
        'cr_hebergement_tva': tva_hebergement,
        'cr_service_ttc': ttc_service,
        'cr_service_ht': ht_service,
        'cr_service_tva': tva_service,
        'cr_total_ttc': total_ttc,
        'cr_total_ht': total_ht,
        'cr_total_tva': total_tva
    })

# Apply the function to each row in the dataframe
df_hebergement_service = df.apply(compute_hebergement_service, axis=1)
print(df_hebergement_service.head())  # Display the first 5 rows of the result
